In [1]:
from utils.data_utils import load_dataset, save_dataset
from utils.functions import load_problem
from utils import load_model

In [2]:
problem = load_problem('tsp')

In [3]:
reviser_path = 'pretrained_LCP/constructions/Reviser-6-FI/reviser_100/epoch-299.pt'
reviser, _ = load_model(reviser_path, is_local=True)
reviser.eval()
reviser.set_decode_type("greedy")

  [*] Loading model from pretrained_LCP/constructions/Reviser-6-FI/reviser_100/epoch-299.pt


In [4]:
dataset = reviser.problem.make_dataset(filename='data/FI_train_tsp/500_FI100_seed1235.pkl', num_samples=1000000, offset=0)

In [5]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=500)

In [7]:
from tqdm import tqdm
import torch
from utils.functions import revision
import copy

original_subtour = torch.arange(100)

get_cost_func = lambda input, pi: problem.get_costs(input, pi, return_local=True)

RG_dataset = []

for batch in tqdm(dataloader):
    decomposed_seeds = batch
    # a = copy.deepcopy(decomposed_seeds[4])
    decomposed_seeds = revision(get_cost_func, reviser, decomposed_seeds, original_subtour)
    # print(decomposed_seeds[4]==a)
    for instance in decomposed_seeds:
        RG_dataset.append(instance)
    # print(len(RG_dataset))


100%|██████████| 2000/2000 [30:34<00:00,  1.09it/s]


In [8]:
len(RG_dataset)

1000000

In [9]:
RG_dataset[0][0],RG_dataset[500000][0]

(tensor([0.0000, 0.0020]), tensor([0.9786, 0.7087]))

In [10]:
new = torch.stack(RG_dataset)

In [11]:
print(new.shape)

torch.Size([1000000, 50, 2])


In [12]:
torch.save(new, 'data/RG_train_tsp/RG100_seed1235.pt')

In [15]:
a = torch.load('data/RG_train_tsp/RG100_seed1235.pt')

In [16]:
a.shape

torch.Size([1000000, 50, 2])